In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import pandas as pd
import pickle

# Leitura dos dados de treino
df_treino = pd.read_csv('./data/Train.csv')

# Inicialização do CountVectorizer
cv = CountVectorizer(lowercase=True, stop_words='english', ngram_range=(1, 1))

# Transformação dos dados de treino
X_treino = cv.fit_transform(df_treino['text'])
y_treino = df_treino['label']

# Inicialização do modelo KNN
knn_model = KNeighborsClassifier()

# Define os hiperparâmetros conforme a parte do código fornecida
parameters = {
    "n_neighbors": [3, 5, 7, 9],
    "weights": ["uniform", "distance"],
    "p": [1, 2],
}

# Procura os melhores hiperparâmetros
grid_search = GridSearchCV(
    knn_model,
    parameters,
    n_jobs=4,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=5),
)
grid_search.fit(X_treino, y_treino)

# Imprimindo todos os parâmetros do melhor modelo
print("Todos os parâmetros do melhor modelo:", grid_search.best_estimator_.get_params())

# Treinamento do modelo com o melhor conjunto de hiperparâmetros
best_knn_model = grid_search.best_estimator_

# Função para carregar e pré-processar os dados IMDB
def load_and_preprocess_IMDB(filename, nrows=None):
    """ load the IMDB data and preprocess it:
            - remove html tags
            - remove ponctuation
            - convert to lower case
            - remove stop words
            - remove numbers
            - remove extra spaces
            - replave words with their root form (stem)
            - replace words with their lemma
        :param filename: 'train' or 'test'
        :param nrows: number of rows to read
        :return: df
    """

    # read the data
    df = pd.read_csv(filename, nrows=nrows)

    # keep a copy of the original text
    df['original_text'] = df['text']

    # remove the html tags
    df['text'] = df['text'].str.replace('<br />', ' ')

    # remove the punctuation and '_' characters
    df['text'] = df['text'].str.replace('[^\w\s]', ' ', regex=True)
    df['text'] = df['text'].str.replace('_', ' ', regex=False)

    # convert to lower case
    df['text'] = df['text'].str.lower()

    # remove the stop words
    from nltk.corpus import stopwords
    stop_words = stopwords.words('english')
    df['text'] = df['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))

    # remove the numbers - test on https://regexr.com
    df['text'] = df['text'].str.replace('\d+', '', regex=True)

    # remove the extra spaces - test on https://regexr.com
    df['text'] = df['text'].str.replace(' +', ' ', regex=True)

    # replace the words with their root form
    from nltk.stem import SnowballStemmer
    stemmer = SnowballStemmer('english')
    df['text'] = df['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

    # replace the words with their lemma
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    df['text'] = df['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

    return df

# Leitura e pré-processamento dos dados de teste
df_teste = load_and_preprocess_IMDB('./data/Test.csv')  # Ajuste o caminho aqui
X_teste = cv.transform(df_teste['text'])  # Transformar os dados de teste

# Avaliação do modelo nos dados de teste
acuracia_teste = best_knn_model.score(X_teste, df_teste['label'])
print(f"Acurácia do KNN nos dados de teste com hiperparâmetros otimizados: {acuracia_teste}")

# Salvando o modelo e o vetorizador no disco
filename_knn_model = 'knn_model.sav'
pickle.dump(best_knn_model, open(filename_knn_model, 'wb'))

filename_vectorizer = 'knn_count_vectorizer.sav'
pickle.dump(cv, open(filename_vectorizer, 'wb'))


Todos os parâmetros do melhor modelo: {'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'metric_params': None, 'n_jobs': None, 'n_neighbors': 9, 'p': 2, 'weights': 'distance'}
Acurácia do KNN nos dados de teste com hiperparâmetros otimizados: 0.5798
